# Ron Low

In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet("yahoo-finance.parquet")

In [3]:
yfinance_metrics = [
    "BTC-CAD_Low"
]

df = df[yfinance_metrics]

In [4]:
# Add an assertion to ensure there are no NA values in the DataFrame
assert df.isnull().sum().sum() == 0, "DataFrame contains NA values"

df

,BTC-CAD_Low
Date,
2014-09-17,497.302246
2014-09-18,452.183655
2014-09-19,419.985870
2014-09-20,427.350739
2014-09-21,430.965698
...,...
2024-03-04,83437.976562
2024-03-05,84514.890625
2024-03-06,80644.625000


In [5]:
import pandas as pd
import numpy as np
import torch

from gluonts.dataset.common import ListDataset
from gluonts.torch.model.tft import TemporalFusionTransformerEstimator
from gluonts.transform.feature import MissingValueImputation

torch.set_float32_matmul_precision('high')
                                   
# Assuming df is your DataFrame with the data
target_column = 'BTC-CAD_Low'  # Replace with your target column name

# Ensure the DataFrame's index is a datetime index and set the frequency explicitly if needed
df.index = pd.to_datetime(df.index)
freq = "D"  # Set the frequency of your data, e.g., 'D' for daily. Adjust as needed.
df = df.asfreq(freq)

# Define the prediction length
prediction_length = 1  # Set your prediction length

# Make sure the lengths match when creating ListDataset
training_data = ListDataset([
    {
        "start": df.index[0],
        "target": df[target_column][:-100]    
    }
], freq=freq)

test_data = ListDataset([
    {
        "start": df.index[-100],
        "target": df[target_column][-100:].values
    }
], freq=freq)

# Initialize the Temporal Fusion Transformer Estimator
estimator = TemporalFusionTransformerEstimator(
    freq=freq,
    prediction_length=prediction_length,
    context_length=60,  # Optional: adjust based on your needs
    num_heads=32,
    hidden_dim=1024,
    variable_dim=1024,
    quantiles=[0.1, 0.5, 0.9],  # Specifying the quantiles for forecasting
    lr=0.001,
    weight_decay=1e-08,
    dropout_rate=0.1,
    patience=10,
    batch_size=128,
    num_batches_per_epoch=100,
    trainer_kwargs={'max_epochs': 1000},  # Adjust 'gpus' based on your setup
)

# Train the model
predictor = estimator.train(training_data)


Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
C:\Users\falty\AppData\Local\Programs\Python\Python312\Lib\site-packages\lightning\pytorch\trainer\connectors\logger_connector\logger_connector.py:67: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
C:\Users\fa

Epoch 0: |                                               | 100/? [00:12<00:00,  7.70it/s, v_num=25, train_loss=3.82e+3]

Epoch 0, global step 100: 'train_loss' reached 3822.65625 (best 3822.65625), saving model to 'C:\\Users\\falty\\Desktop\\GZS\\trading\\options\\tft\\ron\\lightning_logs\\version_25\\checkpoints\\epoch=0-step=100.ckpt' as top 1


Epoch 1: |                                               | 100/? [00:12<00:00,  8.00it/s, v_num=25, train_loss=1.31e+3]

Epoch 1, global step 200: 'train_loss' reached 1309.65308 (best 1309.65308), saving model to 'C:\\Users\\falty\\Desktop\\GZS\\trading\\options\\tft\\ron\\lightning_logs\\version_25\\checkpoints\\epoch=1-step=200.ckpt' as top 1


...

Epoch 484, global step 48500: 'train_loss' was not in top 1


Epoch 485: |                                               | 100/? [00:12<00:00,  8.03it/s, v_num=25, train_loss=8.060]

Epoch 485, global step 48600: 'train_loss' was not in top 1


Epoch 486: |                                                | 22/? [00:02<00:00,  7.72it/s, v_num=25, train_loss=8.060]

C:\Users\falty\AppData\Local\Programs\Python\Python312\Lib\site-packages\lightning\pytorch\trainer\call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [6]:
# Collect actual and predicted values for evaluation, including percentiles
actuals = df[target_column][-prediction_length:].values

mean_predictions = []
p10_predictions = []
p50_predictions = []
p90_predictions = []

for forecast in predictor.predict(test_data):
    p10_predictions.append(forecast.quantile(0.1))
    p50_predictions.append(forecast.quantile(0.5))  # Median
    p90_predictions.append(forecast.quantile(0.9))

# Convert lists to numpy arrays for slicing
p10_predictions = np.array(p10_predictions).flatten()[:prediction_length]
p50_predictions = np.array(p50_predictions).flatten()[:prediction_length]
p90_predictions = np.array(p90_predictions).flatten()[:prediction_length]

In [7]:
last_row = df.iloc[-prediction_length:, ]
last_row

,BTC-CAD_Low
Date,
2024-03-08,88748.6875


In [8]:
print(p10_predictions, p50_predictions, p90_predictions)

[88294.445] [88318.22] [88342.734]


In [9]:
# Function to calculate sMAPE
def calculate_smape(forecasts, actuals):
    return 100 * np.mean(2 * np.abs(forecasts - actuals) / (np.abs(actuals) + np.abs(forecasts)))

# Calculate standard evaluation metrics for mean predictions
mae = np.mean(np.abs(p50_predictions - actuals))
mape = np.mean(np.abs((p50_predictions - actuals) / actuals)) * 100
smape = calculate_smape(p50_predictions, actuals)

# Calculate the percentage of actuals within the 10th to 90th percentile range
within_range = np.sum((actuals >= p10_predictions) & (actuals <= p90_predictions)) / len(actuals) * 100

print("Evaluation Metrics for Mean Predictions:")
print(f"MAE: {mae:.2f}")
print(f"MAPE: {mape:.2f}%")
print(f"sMAPE: {smape:.2f}%")
print(f"Percentage of Actuals within P10-P90 Interval: {within_range:.2f}%")


Evaluation Metrics for Mean Predictions:
MAE: 430.47
MAPE: 0.49%
sMAPE: 0.49%
Percentage of Actuals within P10-P90 Interval: 0.00%


In [10]:
import os
import torch

# Define a path to save the model
model_save_path = 'ron_low_huge.pth'

# Assuming 'predictor' is the trained model from DeepAREstimator
torch.save(predictor, model_save_path)

print(f'Model saved to {model_save_path}')


Model saved to ron_low_huge.pth


In [11]:
import pandas as pd
import numpy as np
from gluonts.dataset.common import ListDataset
# [other imports]

# [Assuming your DataFrame 'df' and the target_column are already defined]
# [Also assuming the estimator and model training are already done and you have 'predictor']

prediction_length = 1  # Forecasting 30 days into the future
context_length = 60  # The length of the history to consider for the prediction

# Rolling window prediction function
def perform_rolling_prediction(df, predictor, target_column, context_length, prediction_length):
    rolling_predictions = []
    prediction_dates = []
    temp_df = df.copy()  # Create a copy of the dataframe to modify

    # Create dates for predictions
    start_prediction_date = temp_df.index.max() + pd.Timedelta(1, unit='D')
    prediction_dates = pd.date_range(start=start_prediction_date, periods=30, freq='D')
    
    for i, prediction_date in enumerate(prediction_dates):
        end_idx = len(temp_df) - prediction_length + i
        test_data = ListDataset([
            {
                "start": temp_df.index[0],
                "target": temp_df[target_column][:end_idx].values
            }
        ], freq='D')

        forecast = next(predictor.predict(test_data))
        predicted_value = forecast.quantile(0.5)[-1]
        rolling_predictions.append(predicted_value)
        
        # Append the predicted value for future rolling windows
        new_row = pd.DataFrame({target_column: [predicted_value]}, index=[prediction_date])
        temp_df = pd.concat([temp_df, new_row])
    
    return prediction_dates, rolling_predictions

# Perform rolling predictions for the next 30 days
prediction_dates, rolling_predictions = perform_rolling_prediction(df, predictor, target_column, context_length, prediction_length)

# Print the rolling predictions
print("Rolling Predictions for the next 30 days:")
for date, prediction in zip(prediction_dates, rolling_predictions):
    print(f"{date.strftime('%Y-%m-%d')}: {prediction}")


Rolling Predictions for the next 30 days:
2024-03-09: 87184.7421875
2024-03-10: 83300.453125
2024-03-11: 82931.9296875
2024-03-12: 83532.6328125
2024-03-13: 86367.1015625
2024-03-14: 84704.15625
2024-03-15: 85655.6484375
2024-03-16: 85643.875
2024-03-17: 86216.234375
2024-03-18: 85878.578125
2024-03-19: 87462.578125
2024-03-20: 84427.3671875
2024-03-21: 82836.71875
2024-03-22: 82807.03125
2024-03-23: 83727.578125
2024-03-24: 83600.53125
2024-03-25: 83759.21875
2024-03-26: 84063.5
2024-03-27: 82959.8515625
2024-03-28: 84148.703125
2024-03-29: 85253.59375
2024-03-30: 85472.171875
2024-03-31: 85572.4921875
2024-04-01: 84990.6015625
2024-04-02: 82599.4140625
2024-04-03: 82974.8125
2024-04-04: 81697.921875
2024-04-05: 81566.71875
2024-04-06: 81383.7265625
2024-04-07: 81259.3515625
